In [4]:
import pandas as pd
from Utils import *
data = pd.read_csv('Tweets/training.1600000.processed.noemoticon.csv', encoding='latin-1', header=None)

In [22]:
from nltk.tokenize import TweetTokenizer
tok = TweetTokenizer(strip_handles=True, reduce_len=True)

In [28]:
import re
def cleaner(text) :
    text = re.sub(r'\s+', ' ', text.strip())
    toks = tok.tokenize(text)
    toks = [t.lower() for t in toks]
    text = ['qqq' if any(char.isdigit() for char in word) else word for word in toks]
    return " ".join(text)

In [31]:
strings = list(data[5])
target = [0 if x == 0 else 1 for x in list(data[0])]

In [35]:
strings = [cleaner(s) for s in strings]

In [46]:
from sklearn.model_selection import train_test_split
train_idx, test_idx = train_test_split(range(len(strings)), test_size = 0.01, stratify=target, random_state=13020)
target_train = [target[i] for i in train_idx]
train_idx, rem_idx = train_test_split(train_idx, test_size=0.9, stratify=target_train, random_state=12930)

In [47]:
len(train_idx), len(test_idx)

(158400, 16000)

In [48]:
X_train = [strings[i] for i in train_idx]
X_test = [strings[i] for i in test_idx]

y_train = [target[i] for i in train_idx]
y_test = [target[i] for i in test_idx]

In [58]:
import vectorizer
vec = vectorizer.Vectorizer(min_df=3)

In [59]:
vec.fit(X_train + X_test)

In [62]:
vec.seq_text = {}
vec.seq_text['train'] = vec.get_seq_for_docs(X_train)
vec.seq_text['test'] = vec.get_seq_for_docs(X_test)

In [63]:
vec.label = { 'train' : y_train, 'test' : y_test } 

In [66]:
from torchtext.vocab import Vectors, GloVe, CharNGram, FastText
url = 'https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.simple.vec'
vectors = Vectors('wiki.simple.vec', url=url)

In [67]:
vec.word_dim = vectors.dim

In [69]:
import numpy as np
vec.embeddings = np.zeros((len(vec.idx2word), vec.word_dim))
for i, word in vec.idx2word.items() :
    vec.embeddings[i] = vectors[word].numpy()

In [70]:
import pickle
pickle.dump(vec, open('Tweets/vec.p', 'wb'))